Podan je slovar `postaje`, katerega ključi so imena postaj Biciklja, vrednosti pa slovarji z zemljepisnimi koordinatami postajališč, kapaciteto (število stojal) in trenutnim številom koles na postaji. (Glej slovar v testih.)

Podana je tudi funkcija `razdalja_geo(lat1, lon1, lat2, lon2)`, ki za podani par zemljepisnih koordinat vrne razdaljo med točkama v kilometrih. (Funkcija deluje pravilno samo za kratke razdalje.)

### 0. Razdalje

Napišite funkcijo `razdalja(postaja1, postaja2)`, ki prejme imeni postaj in vrne razdaljo med njima. (Funkcija je vredna 0 točk. Pišete jo za ogrevanje in rabo v naslednjih nalogah.)

#### Rešitev

Nič takega, seveda, samo postaji je potrebno vzeti iz slovarja in poklicati funkcijo `razdalja_geo`. Recimo tako.

In [1]:
def razdalja(p1, p2):
    postaja1 = postaje[p1]
    postaja2 = postaje[p2]
    return razdalja_geo(postaja1['latitude'], postaja1['longitude'],
                        postaja2['latitude'], postaja2['longitude'])

### 1. Najbližje 3 [tema: seznami]

Avtor izpita ni uporabnik Biciklja (enkrat je bilo dovolj), ve pa, da na določeni postaji včasih ne moreš dobiti ali oddati kolesa, zato se je potrebno ozreti po bližnjih postajah.

Napišite funkcijo `najblizje3(postaja)`, ki za podano postajo poišče najbližje tri postaje in vrne seznam (treh) parov (razdalja, ime). Seznam naj bo urejen po razdaljah.

Klic `najblizje3("ŽIVALSKI VRT")` vrne `[(0.73687236710574, 'VIŠKO POLJE'), (0.9639016462683163, 'TEHNOLOŠKI PARK'), (1.0760703913614853, 'SOSESKA NOVO BRDO')]`.

#### Rešitev

Najpreprosteje je sestaviti seznam natančno takšnih parov, kot jih zahteva naloga (naloga je sestavljena, kakor je, prav zato, da vam da ta namig!), jih urediti in vrniti prve tri. (Tudi, da mora biti rezultat urejen, je nekaj, kar bi vas lahko napeljalo na pravo pot.)

In [2]:
def najblizje(postaja):
    pari = []
    for ime in postaje:
        if ime != postaja:
            pari.append((razdalja(postaja, ime), ime))
    pari.sort()
    return pari[1:4]

No, ne prvih treh: prvi element je vedno izhodiščna postaja in jo preskočimo, zato vzamemo `[1:4]`.

Še hitreje gre, če poznamo izpeljane sezname.

In [3]:
def najblizje3(postaja):
    return sorted((razdalja(postaja, name), name) for name in postaje)[1:4]

Če se ne domislimo urejanja, so stvari seveda bolj "zabavne". No, ni tako hudo: trikrat zapored poiščemo najbližjo postajo in jo dodamo v seznam.

In [4]:
def najblizje(postaja):
    moznosti = list(postaje)
    moznosti.remove(postaja)
    naj3 = []
    for i in range(3):
        naj = None
        for ime in moznosti:
            if naj is None or razdalja(postaja, ime) < razdalja(postaja, naj):
                naj = ime
        naj3.append((razdalja(postaja, naj), naj))
        moznosti.remove(naj)
    return naj3

Ni tako lepo, spet pa ni neka znanstvena fantastika.

### 2. Angelčini jezdeci [tema: rekurzija]

Angelca z Oddelka za gozdarske dejavnosti in motorni promet ima rada zgodovinske filme in ve, da so imeli včasih postaje, na katerih so po določeni razdalji menjali konje. Ker je Bicikelj, tako meni Angelca, isto, bodo po novem omejili dolžino poti, ki jo je dovoljeno opraviti z izposojenim kolesom: razdalja med postajama, na katerih bo kolo izposojeno in vrnjeno, ne bo smela preseči neke predpisane razdalje.
Za daljše razdalje se bo potrebno voziti od postajališča do postajališča in menjati kolesa: nekdo bo najel kolo na Cankarjevi ulici, se z njim peljal do Bavarskega dvora, ga tam vrnil in vzel naslednjega, s katerim bo šel do Vilharjeve ceste ... in tako naprej. Angelca pravi, da bo to ful fino.

Če bo razdalja prekratka, bo zemljevid razpadel na "otoke" medsebojno dosegljivih postaj – skupine postaj, za katere velja, da ne obstaja nobena postaja, ki bi jim bila bližja od predpisane razdalje.

Napišite funkcijo `dosegljive(postaja, razdalja, otok)`, ki prejme ime neke postaje, največjo dovoljeno razdaljo in prazno množico otok. Funkcija ne vrne ničesar, pač pa mora v podano množico otok shraniti vse postaje, ki so pri podani omejitvi razdalje neposredno ali posredno, prek drugih postaj, dosegljive z začetne postaje (vključno z začetno postajo).

```
otok = set()
dosegljive("TEHNOLOŠKI PARK", 0.7, otok)
print(otok)
```

izpiše `{'VIŠKO POLJE', 'SOSESKA NOVO BRDO', 'TEHNOLOŠKI PARK'}`.

#### Rešitev

Tole je naloga iz rekurzije, podobna onim z rodbino: z določene postaje so dosegljive vse postaje, ki so dovolj blizu, poleg njih pa postaje, ki so dosegljive s teh postaj.

Razlika je v tem, da moramo rezultat nabirati v zadnjem argumentu. Tudi takšno različico boste našli v rodbinskih zapiskih. Naloga je podana v tej obliki zato, da lažje preverjate, ali ste določeno postajo že obravnavali in dodali. V nasprotnem primeru bi se rekurzija namreč zaciklala.

In [11]:
def dosegljive(postaja, max_razdalja, otok):
    otok.add(postaja)
    for ime in postaje:
        if ime not in mozno and razdalja(ime, postaja) <= max_razdalja:
            dosegljive(ime, max_razdalja, otok)

### 3. Izolirana postaja [tema: malo bolj zavozlane zanke]

Napišite funkcijo `izolirana()`, ki poišče in vrne ime postaje, ki je najbolj oddaljena od postaje, ki ji je najbližja. Za te konkretne postaje je to Zalog, vendar mora biti funkcija napisana splošno.

#### Rešitev

Na vsakem izpitu se trudim dati nalogo, kjer je potrebno malo navozlati neke if-e in for-e. Na tem roku je bila to ta naloga.

Tokrat moram začeti z rešitvijo, ki so se je domislili mnogi študenti:

In [8]:
def izolirana():
    naj_izolirana = None
    naj_razdalja = 0
    for postaja1 in postaje:
        
        min_razdalja = najblizje3(postaja1)[0][0]
        
        if naj_izolirana is None or min_razdalja > naj_razdalja:
            naj_izolirana = postaja1
            naj_razdalja = min_razdalja
    return naj_izolirana

(Vrstico, ki računa `min_razdalja` sem izoliral iz razlogov, ki bodo jasni kasneje.)

Gremo čez vse postaje. Za vsako poiščemo najbližje tri, saj smo si za to pripravili funkcijo. Zanima nas le prva in še v zvezi z njo je razdalja, ne ime, zato `[0][0]`. Nato preverimo, ali je ta razdalja večja od največje doslej. Če je tako, si zapomnimo to razdaljo in tudi postajo.

Na to iznajdljivo rešitev pri sestavljanju naloge nisem pomislil. Predstavljal sem si, da bo videval nekaj takšnega.

In [6]:
def izolirana():
    naj_izolirana = None
    naj_razdalja = 0
    for postaja1 in postaje:
        
        min_razdalja = None
        for postaja2 in postaje:
            if postaja1 != postaja2:
                r = razdalja(postaja1, postaja2)
                if min_razdalja is None or r < min_razdalja:
                    min_razdalja = r
                    
        if naj_izolirana is None or min_razdalja > naj_razdalja:
            naj_izolirana = postaja1
            naj_razdalja = min_razdalja
    return naj_izolirana

Namesto klica `najblizje3` imamo tu šest vrstic - ki v bistvu počno isto.

Seveda obstaja krajša rešitev. (Vedno obstaja krajša rešitev.)

In [7]:
def izolirana():
    return max(
        (min(razdalja(postaja1, postaja2) for postaja2 in postaje if postaja2 != postaja1), postaja1)
        for postaja1 in postaje)[1]

Za vsako `postaja1` sestavimo par `(min(razdalja(postaja1, postaja2) for postaja2 in postaje if postaja2 != postaja1), postaja1)`, torej minimalno razdaljo do katere izmed postaj in ime te postaje. Nato z `max` izberemo "največji par". Ker ta vsebuje razdaljo in ime, vrnemo element z indeksom 1. Ta, ki se mu da poglobiti v to rešitev, bo opazil, da naredi natančno isto, kar smo delali zgoraj, le vsi `if`-i se skrijejo v `min` in `max`.

Mimogrede: rešitev z `najblizje3` je v teoriji najpočasnejša, ker stalno nekaj ureja, in to je počasna operacija. (V drugem letniku boste rekli, da ima ta rešitev časovno zahtevnost $O(n^2\log n)$. Drugi dve sta v teoriji enako hitri ($O(n^2)$). Drugo bi brez večjih težav pospešili, da bi bila še enkrat hitrejša, vendar je to v teoriji isto. V drugem letniku boste (upam) spoznali način, s katero bi nalogo rešili v času, ki bi bil v praksi toliko hiter, kolikor časa potrebujemo za eno samo urejanje.

### 4. Piši - beri [tema: delo za datotekami in nizi]

- Napišite funkcijo `zapisi(ime_datoteke)`, ki v podano datoteko zapiše imena postaj (na 40 mest), ki ji sledi število razpoložljivih koles in število mest. Zapis mora biti urejen po abecedi imen postaj (šumniki pa bodo na koncu; uporabite običajno urejanje v Pythonu) in v spodnji obliki.

```
KONGRESNI TRG-ŠUBIČEVA ULICA             9/20
KOPALIŠČE ILIRIJA                        0/20
KOPALIŠČE KOLEZIJA                      19/20
KOPRSKA ULICA                            3/8 
KOSEŠKI BAJER                           14/20
```

- Napišite funkcijo `preberi(ime_datoteke)`, ki prebere takšno datoteko v slovar. Ključi slovarja so imena postaj, pripadajoče vrednosti pa pari s številom koles in številom mest.

#### Rešitev

Tole je bilo mišljeno kot podarjena naloga. Žal ste imeli nekaj nepredvidenih težav s poravnavanjem številk na desno in levo ter, no, predvideno težavo z razbijanjem vrstice pri branju.

O pisanju nimamo kaj dosti razpravljati. Poglejte znakce za urejanje - `:2` in `:<2`.

In [9]:
def zapisi(ime_datoteke):
    f = open(ime_datoteke, "w", encoding="utf-8")
    for ime in sorted(postaje):
        f.write(f"{ime:40}{postaje[ime]['bikes']:2}/{postaje[ime]['capacity']:<2}\n")

Z branjem najlažje opravimo tako, da celotno vrstico razbijemo glede na presledke, potem pa vse razen zadnjega kosa sestavimo nazaj. (To ne bo delovalo, če bodo v imenih postaj dvojni presledki, vendar ni greh računati, da jih ne bo.) Zadnji element pa razdelimo še glede na `/`.

In [12]:
def preberi(ime_datoteke):
    post = {}
    for vrstica in open(ime_datoteke, encoding="utf-8"):
        b = vrstica.split()
        ime = " ".join(b[:-1])
        kol, kap = b[-1].split("/")
        post[ime] = (kol, kap)
    return post

Če se ne domislimo trika s `split` in `join`, pač poiščemo zadnji presledek. Ime je vse, kar je pred njim (pa še en `strip`, da odluščimo vse dodatne presledke), številki pa sta za njim.

In [13]:
def preberi(ime_datoteke):
    post = {}
    for line in open(ime_datoteke, encoding="utf-8"):
        line = line.strip()
        zadnji_presledek = line.rfind(" ")
        ime = line[:zadnji_presledek].strip()
        kol, kap = line[zadnji_presledek + 1:].split("/")
        post[ime] = (kol, kap)
    return post

### 5. Koordinator [tema: objektno orientirano programiranje]

Napišite razred `Koordinator`.
-	Konstruktor ne prejme argumentov. Shrani lahko, kar želite. Recimo podatke iz postaje. Ali pa tudi ne.
-	`stanje(postaja)` vrne število razpoložljivih koles na podani postaji
-	`odpelji(postaja)` vrne `True`, če je na postaji kako kolo in `False`, če ga ni. Če je, si zapomni, da je na postaji eno kolo manj.
-	`vrni(postaja)` vrne `True`, če je na postaji kako prosto mesto za kolo in `False`, če ga ni. Če je, si zapomni, da je na postaji eno kolo več.

Pomembno: razred ne sme spreminjati vrednosti v (globalnem) slovarju postaje!

#### Rešitev

Pri teh nalogah se je vedno potrebno odločiti, kako bomo shranjevali podatke. Meni je najbolj všeč shraniti kapacitete in število koles v dva slovarja. V spodnji rešitvi sem uporabil izpeljane slovarje; seveda gre tudi z običajno zanko.

Ko je ta odločitev za nami, je vse preprosto.

In [ ]:
class Koordinator:
    def __init__(self):
        self.kapacitete = {ime: p["capacity"] for ime, p in postaje.items()}
        self.kolesa = {ime: p["bikes"] for ime, p in postaje.items()}

    def odpelji(self, ime):
        if self.kolesa[ime] == 0:
            return False
        self.kolesa[ime] -= 1
        return True

    def vrni(self, ime):
        if self.kolesa[ime] == self.kapacitete[ime]:
            return False
        self.kolesa[ime] += 1
        return True

    def stanje(self, ime):
        return self.kolesa[ime]